In [ ]:
#@title sshd & drive
#from google.colab import drive
#drive.mount('/content/drive')

#Setup sshd
!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
!mkdir -p /var/run/sshd
!echo "PermitRootLogin without-password" >> /etc/ssh/sshd_config
!mkdir -p /root/.ssh
!cat gcloud.pub >> /root/.ssh/authorized_keys

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

!chmod 400 ./gcloud.pem 
!apt install bc  # used for math

#! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
#! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

In [ ]:
#@title remote tunnel
public_server = "34.70.114.171" #@param {type:"string"}

!chmod 400 gcloud.pem

!ssh -o StrictHostKeyChecking=no -i /content/gcloud.pem -N -vv \
    -R 3022:localhost:22 -R 39152:localhost:49152 \
    daniele.veri.96@$public_server

In [ ]:
#@title ngrok (unused)

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

#Create ngrok tunnel
get_ipython().system_raw(f'./ngrok start -config ./drive/MyDrive/ngrok-config.yaml ssh nvidia-prof &')
#Get public address
! curl -s http://localhost:4040/api/tunnels

In [ ]:
#@title strong scaling efficiency plots
import matplotlib.pyplot as plt
import seaborn as sns

x=range(1,17)
y = [
7.772,
3.428,
2.974,
1.990,
1.710,
1.736,
1.568,
1.452,
1.442,
1.471,
1.339,
1.392,
1.331,
1.556,
1.303,
1.250]

strong_eff = []
for i,_ in enumerate(y):
    strong_eff.append(y[0]/y[i]/(i+1))

sns.set_style("darkgrid")
plt.figure(figsize=(20,12))
plt.xlabel('thread', fontsize=18)
plt.ylabel('E(p)', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(ymin=0, ymax=1.2)
plt.scatter(x=x, y=strong_eff, s=200)

In [ ]:
#@title weak scaling efficiency plot
y = [
1.841,
2.027,
2.588,
2.800,
2.529,
2.718,
3.289,
3.370,
3.396,
4.025,
3.844,
3.850,
4.740,
4.590,
5.083,
4.947]

nn = [
300000,
599798,
899596,
1199394,
1499192,
1798990,
2098788,
2398586,
2698384,
2998182,
3297980,
3597778,
3897576,
4197374,
4497172,
4796970]

weak_eff = []
for i,_ in enumerate(y):
    weak_eff.append(y[0]/y[i])

plt.figure(figsize=(20,12))
plt.xlabel('thread', fontsize=18)
plt.ylabel('W(p)', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(ymin=0, ymax=1.2)
plt.scatter(x=x, y=weak_eff, s=200)

axes1 = plt.gca()
axes2 = axes1.twiny()
axes2.grid(False)
axes2.ticklabel_format(style='plain')
axes2.set_xticks(nn, True)
axes2.tick_params(axis='both', which='major', labelsize=16)
axes2.set_xlabel("N",{'size':18})

In [ ]:
#@title throughput plot
import numpy as np
import matplotlib.ticker as ticker

yv100 = np.array([
1477974.238,
3071076.251,
5749762.208,
11247410.276,
18659374.578,
29790015.829,
43470186.060,
56616276.753,
73137403.969,
85208427.311]) / 1e6

yp100 = np.array([
1468330.447,
3137287.295,
6110758.476,
11319781.314,
19228862.039,
31043593.773,
43958682.319,
58517871.195,
72335361.501,
83215989.725]) / 1e6

ymx130 = np.array([
1506985.944,
2893537.917,
5539729.336,
6451165.031,
12675292.785,
15759051.858,
19021041.053,
23409526.232,
22043915.717]) / 1e6

nn = [
2000,
4000,
8000,
16000,
32000,
64000,
128000,
256000,
512000,
1024000]

plt.figure(figsize=(20,12))
ax = plt.gca()
plt.xlabel('N', fontsize=18)
plt.ylabel('millions elements / s', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(ymin=0, ymax=125)
plt.scatter(x=nn, y=yv100, s=200, marker='o')
plt.scatter(x=nn, y=yp100, s=200, marker='o')
plt.scatter(x=nn[:-1], y=ymx130, s=200, marker='o')
plt.legend(['Tesla V100-SXM2','Tesla P100-PCIE','GeForce MX130'])
ax.set_xscale("log")
ax.set_xticks(nn)
ax.get_xaxis().set_major_formatter(ticker.ScalarFormatter())
a=ax.get_xticks().tolist()
ax.set_xticklabels(a)
